# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.118995e+02     1.517328e+00
 * time: 0.3630189895629883
     1     1.010244e+01     8.194572e-01
 * time: 2.397813081741333
     2    -1.185528e+01     9.689988e-01
 * time: 3.0714640617370605
     3    -3.438097e+01     7.044177e-01
 * time: 4.0803539752960205
     4    -4.787822e+01     5.438848e-01
 * time: 5.058104038238525
     5    -5.726063e+01     2.050181e-01
 * time: 6.108274936676025
     6    -6.002050e+01     1.379188e-01
 * time: 6.801673889160156
     7    -6.102367e+01     5.739620e-02
 * time: 7.451864957809448
     8    -6.139028e+01     5.100622e-02
 * time: 8.129025936126709
     9    -6.163034e+01     4.091542e-02
 * time: 8.805491924285889
    10    -6.182391e+01     2.816795e-02
 * time: 9.479391098022461
    11    -6.198271e+01     2.188116e-02
 * time: 10.151894092559814
    12    -6.205737e+01     1.784241e-02
 * time: 10.818825960159302
    13    -6.211785e+01     1.735281e-02
 * time: 11.498959064483643
 

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671077
    AtomicLocal         -18.8557699
    AtomicNonlocal      14.8522652
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485382 
    Xc                  -19.3336824

    total               -62.261666459059
